In [1]:
!pip install tensorflow==2.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 56.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uninstalling flatbuffers-24.3.25:
      Successfully uninstalled flatbuffers-24.3.25
  Attempting uninstall: ten

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt
import pandas as pd
import random
import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [2]:
MAX_SEQUENCE_LENGTH = 128
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# Load files
train_pairs = pd.read_csv('drive/MyDrive/train_pairsS2S.csv')
valid_pairs = pd.read_csv('drive/MyDrive/valid_pairsS2S.csv')
test_pairs = pd.read_csv('drive/MyDrive/test_pairsS2S.csv')

Mounted at /content/drive


In [4]:
# Initialize an empty list to store labeled data
labeled_train = []

# Iterate through each tuple in 'data'
for row in train_pairs.text_pairs_dict:
    # Randomly select either src_raw or tgt_raw
    row = eval(row)

    # Append the tuple containing the selected text and its label to 'labeled_data'
    labeled_train.append((row[0], 0))
    labeled_train.append((row[1], 1))

# Shuffle the labeled_data
random.shuffle(labeled_train)

# Convert to DataFrame
data_train = labeled_train
df_train = pd.DataFrame(data_train, columns=['text', 'label'])
y_train = df_train['label']

In [7]:
df_train = df_train[:130000]
y_train = df_train['label']

In [6]:
# Initialize an empty list to store labeled data
labeled_val = []

# Iterate through each tuple in 'data'
for row in valid_pairs.text_pairs_dict:
    # Randomly select either src_raw or tgt_raw
    row = eval(row)

    # Append the tuple containing the selected text and its label to 'labeled_data'
    labeled_val.append((row[0], 0))
    labeled_val.append((row[1], 1))

# Shuffle the labeled_data
random.shuffle(labeled_val)

# Convert to DataFrame
data_val = labeled_val
df_val = pd.DataFrame(data_val, columns=['text', 'label'])
y_val = df_val['label']

In [5]:
# Initialize an empty list to store labeled data
labeled_test = []

# Iterate through each tuple in 'data'
for row in test_pairs.text_pairs_dict:
    # Randomly select either src_raw or tgt_raw
    row = eval(row)

    # Append the tuple containing the selected text and its label to 'labeled_data'
    labeled_test.append((row[0], 0))
    labeled_test.append((row[1], 1))

# Shuffle the labeled_data
random.shuffle(labeled_test)

# Convert to DataFrame
data_test = labeled_test
df_test = pd.DataFrame(data_test, columns=['text', 'label'])
y_test = df_test['label']

In [ ]:
print(f"{len(df_train.text)} training sentences")
print(f"{len(data_val)} training sentences")
print(f"{len(data_test)} test sentences")

130000 training sentences
54440 training sentences
54440 test sentences


In [8]:
train_examples_str = df_train['text'].values.tolist()
val_examples_str = df_val['text'].values.tolist()
test_examples_str = df_test['text'].values.tolist()

bert_train_tokenized = bert_tokenizer(train_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_train_inputs = [bert_train_tokenized.input_ids,
                     bert_train_tokenized.token_type_ids,
                     bert_train_tokenized.attention_mask]
bert_train_labels = np.array(y_train)

bert_val_tokenized = bert_tokenizer(val_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_val_inputs = [bert_val_tokenized.input_ids,
                     bert_val_tokenized.token_type_ids,
                     bert_val_tokenized.attention_mask]
bert_val_labels = np.array(y_val)

bert_test_tokenized = bert_tokenizer(test_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_test_inputs = [bert_test_tokenized.input_ids,
                     bert_test_tokenized.token_type_ids,
                     bert_test_tokenized.attention_mask]
bert_test_labels = np.array(y_test)



In [9]:
def create_bert_cnn_model(bert_cnn_model,
                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                          num_filters = [64, 64, 64],
                          kernel_sizes = [3, 5, 7],
                          dropout = 0.5,
                          hidden_size = 100,
                          learning_rate=0.00005):
    """
    Build a  classification model with BERT, where you apply CNN layers  to the BERT output
    """

    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_cnn_model(bert_inputs)

    tokens = bert_out[0]

    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(tokens)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)

    last_hidden_output = keras.layers.Dense(hidden_size, activation='relu')(conv_output)
    last_hidden_output = keras.layers.Dropout(rate=dropout)(last_hidden_output)

    cnn_prediction = keras.layers.Dense(1, activation='sigmoid')(last_hidden_output)

    classification_model = keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=cnn_prediction)

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics='accuracy')

    return classification_model

In [ ]:
checkpoint_dir = 'drive/MyDrive/'
checkpoint_filepath = checkpoint_dir + 'bert_cnn_weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True)

In [10]:
bert_cnn_model = create_bert_cnn_model(TFBertModel.from_pretrained('bert-base-cased'))
#bert_cnn_model.load_weights("drive/MyDrive/bert_cnn_weights.01-0.68.hdf5")

In [ ]:

bert_cnn_history = bert_cnn_model.fit(bert_train_inputs,
                            bert_train_labels,
                            validation_data=(bert_val_inputs, bert_val_labels),
                            batch_size=64,
                            epochs=2,
                            callbacks=[model_checkpoint_callback]
                            )

Epoch 1/2


2032/2032 [==============================] - 20047s 10s/step - loss: 0.5806 - accuracy: 0.6844 - val_loss: 0.5714 - val_accuracy: 0.6905
Epoch 2/2
2032/2032 [==============================] - 19972s 10s/step - loss: 0.5340 - accuracy: 0.7210 - val_loss: 0.5719 - val_accuracy: 0.6872


In [ ]:
bert_cnn_model.save("drive/MyDrive/bert_cnn_bias2.keras")

In [ ]:
y_pred = bert_cnn_model.predict(bert_test_inputs)
y_pred_labels = np.where(y_pred > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# Compute various evaluation metrics
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels)
recall = recall_score(y_test, y_pred_labels)
f1 = f1_score(y_test, y_pred_labels)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_labels)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.6912564290962527
Precision: 0.6602141933895488
Recall: 0.7881337252020573
F1-score: 0.7185249690189905
Confusion Matrix:
[[16179 11041]
 [ 5767 21453]]


In [11]:
bert_cnn_model.load_weights("drive/MyDrive/bert_cnn_weights.02-0.69.hdf5")

In [14]:
df_source_test = df_test[df_test['label'] == 0]
df_target_test = df_test[df_test['label'] == 1]

In [15]:
source_str = df_source_test['text'].values.tolist()


bert_source_tokenized = bert_tokenizer(source_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_source_inputs = [bert_source_tokenized.input_ids,
                     bert_source_tokenized.token_type_ids,
                     bert_source_tokenized.attention_mask]

In [16]:
source_pred = bert_cnn_model.predict(bert_source_inputs)

851/851 [==============================] - 2936s 3s/step


In [20]:
bias_score_source = (len(source_str) - np.sum(source_pred)) / len(source_str)

In [21]:
bias_score_source

0.6054395464042983

In [22]:
target_str = df_target_test['text'].values.tolist()


bert_target_tokenized = bert_tokenizer(target_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_target_inputs = [bert_target_tokenized.input_ids,
                     bert_target_tokenized.token_type_ids,
                     bert_target_tokenized.attention_mask]

In [23]:
target_pred = bert_cnn_model.predict(bert_target_inputs)

851/851 [==============================] - 2948s 3s/step


In [24]:
bias_score_target = (27220 - np.sum(target_pred)) / 27220
bias_score_target

0.3761608249908156

In [25]:
test_t5_sequences1 = pd.read_csv('drive/MyDrive/test_t5_sequences.csv')
test_t5_sequences2 = pd.read_csv('drive/MyDrive/test_t5_sequences2.csv')
test_t5_sequences3 = pd.read_csv('drive/MyDrive/test_t5_sequences3.csv')

In [26]:
test_t5_sequences = test_t5_sequences1.iloc[:, 1].to_list() + test_t5_sequences2.iloc[:, 1].to_list() + test_t5_sequences3.iloc[:, 1].to_list()

In [27]:
bert_t5_tokenized = bert_tokenizer(test_t5_sequences,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_t5_inputs = [bert_t5_tokenized.input_ids,
                     bert_t5_tokenized.token_type_ids,
                     bert_t5_tokenized.attention_mask]

In [28]:
t5_pred = bert_cnn_model.predict(bert_t5_inputs)

851/851 [==============================] - 2932s 3s/step


In [29]:
bias_score_t5 = (27220 - np.sum(t5_pred)) / 27220
bias_score_t5

0.39120842613427625

In [30]:
test_s2s_sequences1 = pd.read_csv('drive/MyDrive/test_seq2seq_sequences.csv')
test_s2s_sequences2 = pd.read_csv('drive/MyDrive/test_seq2seq_sequences2.csv')
test_s2s_sequences3 = pd.read_csv('drive/MyDrive/test_seq2seq_sequences3.csv')

In [31]:
test_s2s_sequences = test_s2s_sequences1.iloc[:, 1].to_list() + test_s2s_sequences2.iloc[:, 1].to_list() + test_s2s_sequences3.iloc[:, 1].to_list()

In [32]:
bert_s2s_tokenized = bert_tokenizer(test_s2s_sequences,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_s2s_inputs = [bert_s2s_tokenized.input_ids,
                     bert_s2s_tokenized.token_type_ids,
                     bert_s2s_tokenized.attention_mask]

In [33]:
s2s_pred = bert_cnn_model.predict(bert_s2s_inputs)

329/329 [==============================] - 1146s 3s/step


In [34]:
np.sum(s2s_pred)

5993.3516

In [35]:
bias_score_s2s = (len(test_s2s_sequences) - np.sum(s2s_pred)) / len(test_s2s_sequences)
bias_score_s2s

0.4292046130952381